In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import seaborn as sn
try:
    import deephyper
    print(deephyper.__version__)
except (ImportError, ModuleNotFoundError):
    !pip install deephyper

try:
    import ray
except (ImportError, ModuleNotFoundError):
    !pip install ray

0.7.0


In [16]:
(X_train, Y_train), (X_val_test, Y_val_test) = tf.keras.datasets.mnist.load_data()

In [17]:
X_train = X_train / 255.0
X_val, X_test = np.split(X_val_test, 2)
Y_val, Y_test = np.split(Y_val_test, 2)

In [18]:
def encode_numerical_feature(data):
    # Create a Normalization layer for our feature
    normalizer = tf.keras.layers.Normalization()

    # Learn the statistics of the data
    normalizer.adapt(data)

    # Normalize the input feature
    normalized_ds = normalizer(data)
    
    return normalized_ds

In [19]:
def np_to_dataset(X_train, train_labels, test_examples, test_labels, val_examples, val_labels):
    X_train = encode_numerical_feature(X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
    X_test = encode_numerical_feature(test_examples.reshape(test_examples.shape[0], test_examples.shape[1]*test_examples.shape[2]))
    X_val = encode_numerical_feature(val_examples.reshape(val_examples.shape[0], val_examples.shape[1]*val_examples.shape[2]))
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, train_labels))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, test_labels))
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, val_labels))
    return train_dataset, test_dataset, val_dataset

In [7]:
#  X_train = encode_numerical_feature(X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]))
X_train.shape
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))

In [8]:
train_dataset = train_dataset.shuffle(60000).batch(32)


In [9]:
for data in train_dataset:
    print(data)

(<tf.Tensor: shape=(32, 28, 28), dtype=float64, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0.,

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [20]:
def batch_dataset(config: dict, X_train=X_train, Y_train=Y_train, X_test=X_test, Y_test=Y_test, X_val=X_val, Y_val=Y_val):
    batch_size = config['batch_size']
    train_dataset, test_dataset, val_dataset = np_to_dataset(X_train, Y_train, X_test, Y_test, X_val, Y_val)
    train_dataset = train_dataset.shuffle(60000).batch(batch_size)
    test_dataset = test_dataset.shuffle(5000).batch(batch_size)
    val_dataset = val_dataset.shuffle(5000).batch(batch_size)
    return train_dataset, test_dataset, val_dataset

train_dataset, test_dataset, val_dataset = batch_dataset({'batch_size': 32})

In [21]:
for element in train_dataset:
    x, y = element
    print(f"Shape of x: {x.shape}, Shape of y: {y.shape}")

Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32, 784), Shape of y: (32,)
Shape of x: (32,

2024-06-28 15:05:20.966215: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [22]:
config = {
        'batch_size': 36,
        'patience': 10,
        'scheduler_factor': 0.8,
        'dropout_rate': 0.5,
        'units1': 128,
        'units2': 64,
        'units3': 32,
        'units4': 10,
        'units5': 32,
        'units6': 16,
        'activation1': 'relu',
        'activation2': 'sigmoid',
        'activation3': 'sigmoid',
        'activation4': 'softmax',
        'num_layers1': 2,
        'num_layers2': 2,
        'num_layers3': 2,
        'num_layers4': 2,
        'num_layers5': 2,
        'num_layers6': 2,
        'lr': 0.00005,
        'num_epochs': 5,
        'use_batch_norm': True,
        'optimizer': 'adam'
    }

In [23]:
import tensorflow as tf
import tensorflow.keras.backend as K
import json

def count_params(model: tf.keras.Model) -> dict:
    """Evaluate the number of parameters of a Keras model.

    Args:
        model (tf.keras.Model): a Keras model.

    Returns:
        dict: a dictionary with the number of trainable ``"num_parameters_train"`` and
        non-trainable parameters ``"num_parameters"``.
    """

    def count_or_null(p):
        try:
            return K.count_params(p)
        except:
            return 0

    num_parameters_train = int(
        np.sum([count_or_null(p) for p in model.trainable_weights])
    )
    num_parameters = int(
        np.sum([count_or_null(p) for p in model.non_trainable_weights])
    )
    return {
        "num_parameters": num_parameters,
        "num_parameters_train": num_parameters_train,
    }

In [24]:
def run(config: dict):
    tf.autograph.set_verbosity(0)

    model = keras.Sequential([
        keras.layers.Dense(config["units1"], input_shape=(784,), activation=config["activation1"]),
        keras.layers.Dropout(config["dropout_rate"]),
        keras.layers.Dense(config["units2"], activation=config["activation2"]),
        keras.layers.Dense(config["units3"], activation=config["activation3"]),
        keras.layers.Dense(config["units4"], activation=config["activation4"]),
    ])
    
    model.compile(
        optimizer=config["optimizer"],
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        train_dataset, epochs=config["num_epochs"], validation_data=val_dataset, verbose=0
    )

    objective = history.history["val_accuracy"][-1]
    metadata = {
        "loss": history.history["loss"],
        "val_loss": history.history["val_loss"],
        "accuracy": history.history["accuracy"],
        "val_accuracy": history.history["val_accuracy"],
    }

    metadata = {k:json.dumps(v) for k,v in metadata.items()}
    metadata.update(count_params(model))

    return {"objective": objective, "metadata": metadata}

In [25]:
run(config)

{'objective': 0.9462000131607056,
 'metadata': {'loss': '[0.546058177947998, 0.2649693191051483, 0.22535671293735504, 0.20461340248584747, 0.18768653273582458]',
  'val_loss': '[0.28909215331077576, 0.2217503935098648, 0.2012023627758026, 0.19456231594085693, 0.18914824724197388]',
  'accuracy': '[0.8684166669845581, 0.9235000014305115, 0.9339166879653931, 0.9381666779518127, 0.9443166851997375]',
  'val_accuracy': '[0.9174000024795532, 0.9359999895095825, 0.9423999786376953, 0.9441999793052673, 0.9462000131607056]',
  'num_parameters': 0,
  'num_parameters_train': 111146}}

In [26]:
from deephyper.problem import HpProblem
# from deephyper.search.nas.model.space.structure import KerasStructure
# Define the search space
problem = HpProblem()

problem.add_hyperparameter((64, 256), "units1", default_value=128)
problem.add_hyperparameter((32, 128), "units2", default_value=64)
problem.add_hyperparameter((16, 64), "units3", default_value=32)
problem.add_hyperparameter((4, 32), "units4", default_value=10)
problem.add_hyperparameter(['relu', 'sigmoid'], "activation1", default_value='relu')
problem.add_hyperparameter(['relu', 'sigmoid'], "activation2", default_value='sigmoid')
problem.add_hyperparameter(['relu', 'sigmoid'], "activation3", default_value='sigmoid')
problem.add_hyperparameter(['softmax', 'sigmoid'], "activation4", default_value='softmax')
problem.add_hyperparameter(['adam', 'sgd'], "optimizer", default_value='adam')
problem.add_hyperparameter((0.0, 0.6), "dropout_rate", default_value=0.5)
problem.add_hyperparameter((10, 100), "num_epochs", default_value=50)

problem


TypeError: Argument 'default_value' has incorrect type (expected float, got int)

In [71]:
from tensorflow.python.client import device_lib


def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == "GPU"]


n_gpus = len(get_available_gpus())
if n_gpus > 1:
    n_gpus -= 1

is_gpu_available = n_gpus > 0

if is_gpu_available:
    print(f"{n_gpus} GPU{'s are' if n_gpus > 1 else ' is'} available.")
else:
    print("No GPU available")

No GPU available


In [72]:
import ray

# We launch the Ray run-time depending of the detected local ressources
# and execute the `run` function with the default configuration
# WARNING: in the case of GPUs it is important to follow this scheme
# to avoid multiple processes (Ray workers vs current process) to lock
# the same GPU.
if is_gpu_available:
    if not(ray.is_initialized()):
        ray.init(num_cpus=n_gpus, num_gpus=n_gpus, log_to_driver=False)

    run_default = ray.remote(num_cpus=1, num_gpus=1)(run)
    out = ray.get(run_default.remote(problem.default_configuration))
else:
    if not(ray.is_initialized()):
        ray.init(num_cpus=1, log_to_driver=False)
    run_default = run
    out = run_default(problem.default_configuration)

objective_default = out["objective"]
metadata_default = out["metadata"]

print(f"Accuracy Default Configuration:  {objective_default:.3f}")

print("Metadata Default Configuration")
for k,v in out["metadata"].items():
    print(f"\t- {k}: {v}")


Accuracy Default Configuration:  0.961
Metadata Default Configuration
	- loss: [0.556139349937439, 0.26523062586784363, 0.22398705780506134, 0.20244725048542023, 0.18620677292346954, 0.17843766510486603, 0.17001232504844666, 0.16205507516860962, 0.1529603898525238, 0.14865443110466003, 0.1474284678697586, 0.14446406066417694, 0.14290358126163483, 0.13738416135311127, 0.13506880402565002, 0.1328459084033966, 0.1317557841539383, 0.1269463747739792, 0.12993723154067993, 0.12548573315143585, 0.12637391686439514, 0.12008202821016312, 0.12110385298728943, 0.11567731946706772, 0.11737019568681717, 0.11522474139928818, 0.11346306651830673, 0.11660651117563248, 0.11318917572498322, 0.1133587509393692, 0.11491386592388153, 0.1137080043554306, 0.11289728432893753, 0.11131525784730911, 0.11370473355054855, 0.10763152688741684, 0.11131879687309265, 0.1089775487780571, 0.10953006893396378, 0.10675045102834702, 0.10727628320455551, 0.10677415877580643, 0.10711734741926193, 0.10782462358474731, 0.1028

In [22]:
from deephyper.evaluator import Evaluator
from deephyper.evaluator.callback import TqdmCallback

def get_evaluator(run_function):
    # Default arguments for Ray: 1 worker and 1 worker per evaluation
    method_kwargs = {
        "num_cpus": 1,
        "num_cpus_per_task": 1,
        "callbacks": [TqdmCallback()]
    }

    # If GPU devices are detected then it will create 'n_gpus' workers
    # and use 1 worker for each evaluation
    if is_gpu_available:
        method_kwargs["num_cpus"] = n_gpus
        method_kwargs["num_gpus"] = n_gpus
        method_kwargs["num_cpus_per_task"] = 1
        method_kwargs["num_gpus_per_task"] = 1

    evaluator = Evaluator.create(
        run_function,
        method="ray",
        method_kwargs=method_kwargs
    )
    print(f"Created new evaluator with {evaluator.num_workers} worker{'s' if evaluator.num_workers > 1 else ''} and config: {method_kwargs}", )

    return evaluator
evaluator_1 = get_evaluator(run)

Created new evaluator with 1 worker and config: {'num_cpus': 1, 'num_cpus_per_task': 1, 'callbacks': [<deephyper.evaluator.callback.TqdmCallback object at 0x7fedc956e470>]}


In [105]:
evaluator = Evaluator.create(
    run,
    method="ray",

    method_kwargs={
        "address": "auto",
        "num_cpus": n_gpus,
        "num_gpus": n_gpus,
        "num_cpus_per_task": 1,
        "num_gpus_per_task": 1,
        "num_workers": 3,
    }
)

In [24]:
from deephyper.search.hps import CBO
search = CBO(
    problem,
    evaluator_1,
    initial_points=[problem.default_configuration]
)

In [25]:
results = search.search(max_evals=25)


InvalidArgumentError: Cannot convert a Tensor of dtype variant to a NumPy array.

In [194]:
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(784,), activation='relu'),
    keras.layers.Dense(64, activation='sigmoid'),
    keras.layers.Dense(32, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_normal, Y_train, epochs=5)

Epoch 1/5
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/victorsu-ortiz/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/b1/89wsk5t517v75qvdm2x4ygwc0000gn/T/ipykernel_9213/3944348463.py", line 14, in <module>
    model.fit(X_train_normal, Y_train, epochs=5)
  File "/Users/victorsu-ortiz/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "/var/folders/b1/89wsk5t517v75qvdm2x4ygwc0000gn/T/__autograph_generated_filezsp_59h1.py", line 15, in tf__train_function
    retval_ = ag__.converted_call(ag__.ld(step_function), (ag__.ld(self), ag__.ld(iterator)), None, fscope)
ValueError: in user code:

    File "/Users/victorsu-ortiz/opt/anaconda3/envs/myenv/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        re

In [19]:
model.evaluate(X_test_flat, Y_test)

313/313 [==============================] - 0s 455us/step - loss: 0.2524 - accuracy: 0.9266


[0.25240427255630493, 0.9265999794006348]

In [4]:
def encode_numerical_feature(feature, name, dataset):
    # Create a Normalization layer for our feature
    normalizer = tf.keras.layers.Normalization()

    # Prepare a Dataset that only yields our feature
    feature_ds = dataset.map(lambda x, y: x[name])
    feature_ds = feature_ds.map(lambda x: tf.expand_dims(x, -1))

    # Learn the statistics of the data
    normalizer.adapt(feature_ds)

    # Normalize the input feature
    encoded_feature = normalizer(feature)
    return encoded_feature

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(32)

# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)


170498071/170498071 [==============================] - 6s 0us/step
Epoch 1/10
1563/1563 [==============================] - 13s 8ms/step - loss: 1.4983 - accuracy: 0.4542 - val_loss: 1.3734 - val_accuracy: 0.5285
Epoch 2/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1482 - accuracy: 0.5930 - val_loss: 1.2032 - val_accuracy: 0.5726
Epoch 3/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9954 - accuracy: 0.6498 - val_loss: 0.9850 - val_accuracy: 0.6492
Epoch 4/10
1563/1563 [==============================] - 10s 7ms/step - loss: 0.8893 - accuracy: 0.6873 - val_loss: 0.9504 - val_accuracy: 0.6744
Epoch 5/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8175 - accuracy: 0.7149 - val_loss: 0.9077 - val_accuracy: 0.6829
Epoch 6/10
1563/1563 [==============================] - 13s 8ms/step - loss: 0.7612 - accuracy: 0.7326 - val_loss: 0.8939 - val_accuracy: 0.6908
Epoch 7/10
1563/1563 [==============================] - 11s 7ms